In [8]:
import random
import csv
try:
    import urllib.request as urllib2
except ImportError:
    import urllib2
from deap import base
from deap import creator
from deap import tools
from deap import algorithms
import pandas as pd
import numpy as np
#import QSTK.qstkfeat.featutil as ftu
from pyalgotrade import strategy
from pyalgotrade import barfeed
from pyalgotrade import bar
from pyalgotrade.technical import ma as pytechma
from pyalgotrade import plotter
from pyalgotrade.stratanalyzer import returns
from pyalgotrade.stratanalyzer import sharpe
from pyalgotrade.utils import stats as pytechstats
import datetime as dt
import os

In [9]:
# Change this to your directory structure

download_dir = '/Users/shadmanspc/Desktop/ADA/A2'

#'Specify the stock name in an array 

#Can do for any number of stocks just append the stock name in the stock array and download it
#it from a yahoo finance accordingly
#I took training from 2014-1-1 to 2018-12-31.

stocks = ['BHP.AX']

# keys from yahoo are different from what qstk library expects
#
yahoo_keys = ['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
qstk_keys = ['open', 'high', 'low', 'close', 'volume', 'actual_close']
#Keep track of the stocks and its data
raw_training_data = {}
# cloned for iteration, will remove those stocks from master stocks list if

# Initially designed for fetching direcrly from url but apparently everything has been deprecated.

stocks_clone = [s for s in stocks]

for s in stocks_clone:
    #Change the filename to stockname_training.csv
  
    file_name = download_dir + "/"+ s + '_training'+'.csv'
    print(file_name)
    if(os.path.isfile(file_name)) :
        raw_training_data[s] = pd.read_csv(file_name)
        
#I try to mimick the qstk library but this no longer available so I manipulate the data to my model
#to the way I believe qstk library would work
        

def sanitizedataforqstk(frame):
    data = {}
    for k in yahoo_keys:
        key_df = pd.DataFrame()
        for s in stocks:
            dates = np.array([dt.datetime.strptime(t, "%Y-%m-%d") for t in frame[s]['Date']])
            key_df[s] = pd.Series(data=frame[s][k].values, index=dates).sort_index(ascending=True)
        data[k] = key_df

    # sanitize to use the earlier keys
    data[qstk_keys[0]] = data[yahoo_keys[0]]
    data[qstk_keys[1]] = data[yahoo_keys[1]]
    data[qstk_keys[2]] = data[yahoo_keys[2]]
    data[qstk_keys[3]] = data[yahoo_keys[3]]
    data[qstk_keys[4]] = data[yahoo_keys[4]]
    data[qstk_keys[5]] = data[yahoo_keys[5]]
    del data[yahoo_keys[0]]
    del data[yahoo_keys[1]]
    del data[yahoo_keys[2]]
    del data[yahoo_keys[3]]
    del data[yahoo_keys[4]]
    del data[yahoo_keys[5]]
    for key in qstk_keys:
        data[key] = data[key].fillna(method='ffill')
        data[key] = data[key].fillna(method='bfill')
        data[key] = data[key].fillna(1.0)

    return data

training_data = sanitizedataforqstk(raw_training_data)
#print(training_data)

/Users/shadmanspc/Desktop/ADA/A2/BHP.AX_training.csv


In [10]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates moving average
def featMA( dData, lLookback=30, bRel=True, b_human=False ):
    '''
    @summary: Calculate moving average
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfPrice = dData['close']
    
    dfRet = dfPrice.rolling(lLookback).mean()
    #ts_log.rolling(12).mean()
    
    if bRel:
        dfRet = dfRet / dfPrice
    if b_human:  
        data2 = dfRet * dData['close']
        data3 = pand.DataFrame({"Raw":data2[data2.columns[0]]})
        for sym in dfRet.columns:
            if sym != '$SPX' and sym != '$VIX':
                data3[sym + " Moving Average"] = data2[sym]
                data3[sym] = dData['close'][sym]
        del data3['Raw']
        return data3
    return dfRet

ma_training=featMA(training_data,bRel=False).fillna(0.0)
#Filling not applicable values with mean instead of NA
#ma_training=rsi_training.fillna(ma_training['ASX.AX'].mean())

In [11]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates moving average
def featMA( dData, lLookback=30, bRel=True, b_human=False ):
    '''
    @summary: Calculate moving average
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    
    dfPrice = dData['close']
    
    dfRet = dfPrice.rolling(lLookback).mean()
    #ts_log.rolling(12).mean()
    
    if bRel:
        dfRet = dfRet / dfPrice
    if b_human:  
        data2 = dfRet * dData['close']
        data3 = pand.DataFrame({"Raw":data2[data2.columns[0]]})
        for sym in dfRet.columns:
            if sym != '$SPX' and sym != '$VIX':
                data3[sym + " Moving Average"] = data2[sym]
                data3[sym] = dData['close'][sym]
        del data3['Raw']
        return data3
    return dfRet

ma_training=featMA(training_data,bRel=False).fillna(0.0)
#Filling not applicable values with mean instead of NA
#ma_training=rsi_training.fillna(ma_training['ASX.AX'].mean())

In [ ]:
#This code is copied from https://github.com/tuckerbalch/QSTK/blob/master/qstkfeat/features.py
#This library does is not supported anymore so I do my manipulation
#This code calculates momentum

def returnize0(nds):
    """
    @summary Computes stepwise (usually daily) returns relative to 0, where
    0 implies no change in value.
    @return the array is revised in place
    """
    s= np.shape(nds)
    if len(s)==1:
        nds=np.expand_dims(nds,1)
    #print(s)
    nds[1:, :] = (nds[1:, :] - nds[0:-1]) / abs(nds[0:-1])
    nds[0, :] = np.zeros(nds.shape[1])
    
    #return nds

def featMomentum(dData, lLookback=20, b_human=False ):
    '''
    @summary: N day cumulative return (based on 1) indicator
    @param dData: Dictionary of data to use
    @param lLookback: Number of days to look in the past
    @param b_human: if true return dataframe to plot
    @return: DataFrame array containing values
    '''
    if b_human:
        for sym in dData['close']:
            x=1000/dData['close'][sym][0]
            dData['close'][sym]=dData['close'][sym]*x
        return dData['close']
    dfPrice = dData['close'].copy()
    #print(dfPrice.values)
    
    #Calculate Returns
    returnize0(dfPrice.values)
    
    #Calculate rolling sum
    dfRet = dfPrice.rolling(lLookback).sum()
    #print(dfRet.head(100))
    
    
    return dfRet

mom_training=featMomentum(training_data).fillna(0.0)
#Filling not applicable values with mean instead of NA
#mom_training=rsi_training.fillna(mom_training['ASX.AX'].mean())
#print(mom_training.head(100))